In [1]:
# Import all required libraries
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.data_classes import RadarPointCloud
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, precision=4)  # suppress scientific notation, 4 decimal digits
import pandas as pd
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
from pyquaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points
import copy
import os
from scipy.spatial import cKDTree
import pandas as pd

#Create the nuscene object to read and traverse the data
nusc = NuScenes(version='v1.0-mini', dataroot='../data/sets/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.316 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [2]:
rows = []

for scene in nusc.scene:
    scene_token = scene['token']
    scene_name = scene['name']
    
    sample_token = scene['first_sample_token']
    
    while sample_token:
        sample = nusc.get('sample', sample_token)
        next_token = sample['next']
        
        cam_token = sample['data']['CAM_FRONT']
        cam_data = nusc.get('sample_data', cam_token)

        radar_token = sample['data']['RADAR_FRONT']
        radar_data = nusc.get('sample_data', radar_token)
        
        camera_pose = nusc.get('ego_pose', cam_data['ego_pose_token'])

        radar_pcd_file = os.path.join(nusc.dataroot, radar_data['filename'])
        image_file = os.path.join(nusc.dataroot, cam_data['filename'])

        row = {
            'scene_token': scene_token,
            'scene_name': scene_name,
            'sample_token': sample_token,
            'next_token': next_token,
            'radar_pcd_file': radar_pcd_file,
            'image_file': image_file,
            'camera_pose_x': camera_pose['translation'][0],
            'camera_pose_y': camera_pose['translation'][1],
            'camera_pose_z': camera_pose['translation'][2]
        }

    
        rows.append(row)

        sample_token = next_token  # Move to next sample

df_filenames = pd.DataFrame(rows)

df_filenames.head()

In [3]:
df_filenames.to_excel('Sample-PCD-ImageFileReference.xlsx', index=False)